In [3]:
## BEFORE YOU START... 
#1. install libraries
#2. install chromedriver >> reference :: (https://beomi.github.io/gb-crawling/posts/2017-02-27-HowToMakeWebCrawler-With-Selenium.html)
#3. make a new folder for save book covers

import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import urllib

import pandas as pd 
import numpy as np

import os
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

os.chdir("/Users/janechoi/Desktop/2020-CO-DEEP/CODE1")

print(os.getcwd())

data = pd.read_csv('1114_data.csv')
data.info()

/Users/janechoi/Desktop/2020-CO-DEEP/CODE1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 615075 entries, 0 to 615074
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   student_id   615075 non-null  int64 
 1   book_id      615075 non-null  object
 2   ddc          615075 non-null  object
 3   title        615075 non-null  object
 4   meta_id      615075 non-null  int64 
 5   borrow_type  615075 non-null  object
 6   b_date       615075 non-null  object
 7   r_date       615075 non-null  object
 8   return_type  615075 non-null  object
dtypes: int64(2), object(7)
memory usage: 42.2+ MB


In [4]:
book = data[['book_id', 'ddc', 'title', 'meta_id']]
book
books = book['book_id'].unique()
print(len(books))

196940


In [14]:
bookdb = pd.DataFrame()
    
driver = webdriver.Chrome('/Users/janechoi/Desktop/2020-CO-DEEP/CODE1/chromedriver') 

driver.implicitly_wait(3)
    
for i in tqdm(range(140001, 150000)):
    book_id = books[i]
    url = "https://lib.skku.edu/#/search/total/si?all=1%7Ck%7Ca%7C{}".format(book_id)
    driver.get(url)
    
    try:
        result = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[2]/div/div[2]/div[2]/div[3]/div/ik-total-search[1]/div/div[2]/div/div[2]/ul/li[1]/a[2]')
        result.click()
    
        driver.switch_to.window(driver.window_handles[1])

        book_cover = driver.find_elements_by_css_selector('div.ikc-bookcover')
        book_info = driver.find_element_by_css_selector('div.ikc-biblio-detail')
    
        #GET IMAGE
        img_url = re.findall('(?<=src=").+?(?=")', book_cover[0].get_attribute('outerHTML'))[0]
    
        try :
            img_name = 'cover/' + book_id + '.jpg'
            urllib.request.urlretrieve(img_url,img_name)
            img = 'Y'
        
        except:
            img = 'N'
    
        #GET BOOK INFO
        info_source = book_info.get_attribute('outerHTML')
    
        try :
            info_list1 = re.findall('(?<=서명/저자사항</label><span ik-html-compile="item.content"><span>).+?(?=</span>)', info_source, re.DOTALL)[0]
            book_subject = info_list1.split(' /')[0]
            book_writer = info_list1.split(' /')[1]
        except:
            book_subject = ""
            book_writer = ""
        
        try: 
            info_list2 = re.findall('(?<=발행사항</label><span ik-html-compile="item.content"><span>).+?(?=</span>)', info_source, re.DOTALL)[0]
            book_pub = info_list2.split(", ")[0]
            book_pub = book_pub.split(" :")[1]
            book_age = info_list2.split(", ")[1]
        except:
            book_pub = ""
            book_age = ""
    
        try:
            info_list3 = re.findall('(?<=형태사항</label><span ik-html-compile="item.content"><span>).+?(?=</span>)', info_source, re.DOTALL)[0]
            book_page = info_list3.split(" p.")[0]
        except:
            book_page = ""
   
        bookdb = bookdb.append({"book_id" : book_id, "subject" : book_subject, "writer" : book_writer,
                             "pub" : book_pub, "pub_year" : book_age, "page" : book_page, 'img' : img}, ignore_index=True)
        driver.close()
        driver.switch_to_window(driver.window_handles[0])
    
    except:
        bookdb = bookdb.append({"book_id" : book_id, "subject" : "", "writer" : "",
                              "pub" : "", "pub_year" : "", "page" : "", 'img' :  img}, ignore_index=True)


100%|██████████| 5495/5495 [2:14:31<00:00,  1.47s/it]


In [19]:
bookdb.to_csv('book_150000.csv', index = False)